In [18]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

In [19]:
train_labels = pd.read_csv("train.csv")

### Visualize how the train labels look

In [3]:
a = train_labels[train_labels.eq(1).any(axis=1)]
b = a[a['Modality'] == 'CTA']
b.SeriesInstanceUID.iloc[0]

'1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317'

In [4]:
a['SeriesInstanceUID'].iloc[0]

'1.2.826.0.1.3680043.8.498.10005158603912009425635473100344077317'

In [5]:
train_labels[train_labels.eq(1).any(axis=1)]

,SeriesInstanceUID,PatientAge,PatientSex,Modality,Left Infraclinoid Internal Carotid Artery,Right Infraclinoid Internal Carotid Artery,Left Supraclinoid Internal Carotid Artery,Right Supraclinoid Internal Carotid Artery,Left Middle Cerebral Artery,Right Middle Cerebral Artery,Anterior Communicating Artery,Left Anterior Cerebral Artery,Right Anterior Cerebral Artery,Left Posterior Communicating Artery,Right Posterior Communicating Artery,Basilar Tip,Other Posterior Circulation,Aneurysm Present
2,1.2.826.0.1.3680043.8.498.10005158603912009425...,58,Male,CTA,0,0,0,0,0,0,0,0,0,0,0,0,1,1
8,1.2.826.0.1.3680043.8.498.10022796280698534221...,78,Male,CTA,0,0,0,0,0,1,0,0,0,0,0,0,0,1
9,1.2.826.0.1.3680043.8.498.10023411164590664678...,76,Female,MRA,0,0,0,0,0,1,0,0,0,0,0,0,0,1
10,1.2.826.0.1.3680043.8.498.10030095840917973694...,89,Female,CTA,0,1,0,0,0,0,0,0,0,0,0,0,0,1
12,1.2.826.0.1.3680043.8.498.10034081836061566510...,54,Male,CTA,0,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4394,1.2.826.0.1.3680043.8.498.99800061424469215274...,47,Female,CTA,0,0,0,0,0,0,0,0,0,1,0,0,0,1
4395,1.2.826.0.1.3680043.8.498.99804081131933373817...,57,Female,MRI T2,0,0,1,0,0,0,0,0,0,0,0,0,0,1
4396,1.2.826.0.1.3680043.8.498.99887675554378211308...,53,Female,CTA,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4397,1.2.826.0.1.3680043.8.498.99892390884723813599...,68,Female,CTA,0,0,0,0,1,1,0,0,0,0,0,0,0,1


In [6]:
train_labels.Modality

0              MRA
1              MRA
2              CTA
3              MRA
4              MRA
           ...    
4400    MRI T1post
4401           MRA
4402           CTA
4403        MRI T2
4404           CTA
Name: Modality, Length: 4405, dtype: object

In [7]:
train_labels.describe

<bound method NDFrame.describe of                                       SeriesInstanceUID  PatientAge  \
0     1.2.826.0.1.3680043.8.498.10004044428023505108...          64   
1     1.2.826.0.1.3680043.8.498.10004684224894397679...          76   
2     1.2.826.0.1.3680043.8.498.10005158603912009425...          58   
3     1.2.826.0.1.3680043.8.498.10009383108068795488...          71   
4     1.2.826.0.1.3680043.8.498.10012790035410518400...          48   
...                                                 ...         ...   
4400  1.2.826.0.1.3680043.8.498.99915610493694667606...          62   
4401  1.2.826.0.1.3680043.8.498.99920680741054836990...          76   
4402  1.2.826.0.1.3680043.8.498.99953513260518059135...          44   
4403  1.2.826.0.1.3680043.8.498.99982144859397209076...          58   
4404  1.2.826.0.1.3680043.8.498.99985209798463601651...          65   

     PatientSex    Modality  Left Infraclinoid Internal Carotid Artery  \
0        Female         MRA            

In [8]:
classes = train_labels['Modality'].dropna().unique().tolist()
print(classes)

['MRA', 'CTA', 'MRI T2', 'MRI T1post']


In [9]:
train_labels[train_labels['Modality'] == 'MRI T2']['SeriesInstanceUID'].iloc[0]

'1.2.826.0.1.3680043.8.498.10030804647049037739144303822498146901'

### Finding the series with the aneurysm (got this from chatGPT)

In [28]:
def find_folder(root_dir, target_folder_name):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        if target_folder_name in dirnames:
            # os.path.join creates the full path: root_dir + sub_folder_name
            found_path = os.path.join(dirpath, target_folder_name)
            print(f" Found folder at: {found_path}")
            return found_path

    print(f" Folder '{target_folder_name}' not found in '{root_dir}' or its subdirectories.")
    return None

search_start_path = '/Volumes/X9 Pro/aneurysm/series/'
folder_to_find = '1.2.826.0.1.3680043.8.498.11999987145696510072091906561590137848'
found_folder_path = find_folder(search_start_path, folder_to_find)

elements = os.listdir(found_folder_path)
'series/'+ folder_to_find + '/' + elements[0]

 Found folder at: /Volumes/X9 Pro/aneurysm/series/1.2.826.0.1.3680043.8.498.11999987145696510072091906561590137848


'series/1.2.826.0.1.3680043.8.498.11999987145696510072091906561590137848/1.2.826.0.1.3680043.8.498.10015718034063989458448348558269450247.dcm'

In [33]:
elements[0]

'1.2.826.0.1.3680043.8.498.10015718034063989458448348558269450247.dcm'

In [42]:
import pydicom

# use the first filename/path from your elements list
dcm_path = elements[0]
ds = pydicom.dcmread('series/'+ folder_to_find + '/' + elements[0])

print(ds.get("Modality"))

MR


### Visualize how the train localizers look

In [12]:
train_localizers = pd.read_csv("train_localizers.csv")
train_localizers.head(2)

,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery


In [13]:
train_labels.shape ,train_localizers.shape

((4405, 18), (2286, 4))

In [25]:
example = train_localizers[train_localizers['SeriesInstanceUID'] == '1.2.826.0.1.3680043.8.498.11999987145696510072091906561590137848']

In [26]:
example['SOPInstanceUID'].iloc[0]

'1.2.826.0.1.3680043.8.498.87636090859887160412247303157695174056'

In [27]:
example['coordinates']

394    {'x': 345.3863056586271, 'y': 273.92949907235624}
Name: coordinates, dtype: object

# Whatever

In [17]:
## Trying to understand the CTM scans

